In [ ]:
import os
import pandas as pd
import numpy as np
import datetime
import time

In [ ]:
renfe = pd.read_csv("renfe_clean_1.csv")

In [ ]:
renfe.head(3)

### Just keeping the number of the calculated column days to departure

In [ ]:
renfe["days_to_departure"] = renfe["days_to_departure"].apply(lambda x: x[:2])

### Reducing train type categories into 3 big categories

In [ ]:
high_speed = ["AVE", "AVE-TGV"]
long_distance = ["INTERCITY", "ALVIA", "TRENHOTEL", "AV City"] 
regional = ["MD", "LD", "EXPRESS", "REGIONAL"]

In [ ]:
def train_class(x):
    if x in high_speed:
        return "High_speed"
    elif x in long_distance:
        return "Long_distance"
    else:
        return "Regional"

In [ ]:
renfe["train_category"] = renfe["train_type"].apply(train_class)

### Reducing ticket classes into 2 big categories

In [ ]:
renfe.loc[renfe["ticket_class"].str.match("Turista"), "ticket_class_category"] = "Economy"
renfe.loc[renfe["ticket_class"].str.match("Preferente"), "ticket_class_category"] = "First_class"
renfe.loc[renfe["ticket_class"].str.match("Clase"), "ticket_class_category"] = "First_class"

### Reducing fare types into 2 big categories

In [ ]:
fixed = ["Promo", "Promo +", "Adulto", "Mesa", "Grupos Ida"]

In [ ]:
def fare(x):
    if x in fixed:
        return "Fixed"
    else:
        return "Flexible"

In [ ]:
renfe["fare_category"] = renfe["fare"].apply(fare)

### Creating categories for time variables

In [ ]:
timing = renfe["price_check_time"].apply(lambda x: time.strptime(x, "%H:%M:%S"))
timing_2 = renfe["departure_time"].apply(lambda x: time.strptime(x, "%H:%M:%S"))

In [ ]:
renfe["price_check_hour"] = timing.apply(lambda x: x[3])
renfe["departure_hour"] = timing_2.apply(lambda x: x[3])

In [ ]:
def time_window(t):
    if t > 2 and t < 6:
        return "Deep night"
    elif t >= 6 and t < 10:
        return "Morning"
    elif t >= 10 and t < 14:
        return "Late_morning"
    elif t >= 14 and t < 18:
        return "Afternoon"
    elif t >= 18 and t < 22:
        return "Evening"
    else:
        return "Night"

In [ ]:
renfe["price_check_time_window"] = renfe["price_check_hour"].apply(time_window)
renfe["departure_time_window"] = renfe["departure_hour"].apply(time_window)

In [ ]:
renfe.price_check_time_window.value_counts()

### Last categorization adjustments

In [ ]:
weekday = ["Monday", "Tueday", "Wednesday", "Thursday", "Friday"]

In [ ]:
def day(x):
    if x in weekday:
        return "weekday"
    else:
        return "weekend"

In [ ]:
renfe["departure_day"] = renfe["departure_weekday"].apply(day)

In [ ]:
renfe.drop("ticket_class", axis=1, inplace=True)
renfe.drop("fare", axis=1, inplace=True)

In [ ]:
renfe.head(3)

In [ ]:
renfe.to_csv("../Data/renfe_clean_2.csv", index=False)